In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

![对比 VGG 和 NiN 及它们的块之间主要结构差异。](http://d2l.ai/_images/nin.svg)

NiN 块以一个普通卷积层开始，后面是两个1×1的卷积层。这两个1×1卷积层充当带有 ReLU 激活函数的逐像素全连接层。即第一层的卷积窗口形状通常由用户设置，随后的卷积窗口形状固定为1×1 。

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

#NiN块
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        #后两个1*1的卷积层不改变通道数
        nn.Conv2d(out_channels, out_channels, kernel_size=1), 
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), 
        nn.ReLU())

In [4]:
#NiN的模型
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    #全局平均池化层，最后图片大小为1*1
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)，可以直接拿去softmax
    nn.Flatten())

In [13]:
X = torch.randn(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
#训练
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

Mac由于没有GPU，这里通过markdown直接展示结果。

![avatar](https://zh-v2.d2l.ai/_images/output_nin_8ad4f3_42_1.svg)

loss 0.351, &ensp; train acc 0.868, &ensp; test acc 0.867

3215.4 examples/sec on cuda:0